# Neural Network

A neural network is created which can be used for training and testing on trips data.



In [1]:
!pip install tensorflow

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:
path_output=os.path.join(os.getcwd(), "..", "data", "output")

In [4]:
features = pd.read_csv(os.path.join(path_output, "Features.csv"))
features = features.drop("Unnamed: 0", axis=1)

### Take Params from csv

In [5]:
params = pd.read_csv(os.path.join(os.getcwd(), "..", "data", "input", "params.csv")).drop("0", axis=1)
_test_size = params[params["param"]=="test_size"]["value"].values[0]
_random_state = int(params[params["param"]=="random_state"]["value"].values[0])
_epochs = int(params[params["param"]=="epochs"]["value"].values[0])
_validation_size = params[params["param"]=="validation_size"]["value"].values[0]

### Neural Network

In [6]:
def train_neural_network(X_train_scaled, y_train, on, hex_size):
    """
    Train Neural Network Model

    Train and save a Neural Network model.
    The network has the following properties:
        - three hidden layer
        - 50 epochs
        - activation function is relu
        - dimension of input and hidden layer is 36
        - dimension of output layer is 1
        - dropout is not used
    Then evaluate the error metrics by another method.

    Args:
        X_train_scaled (DataFrame):   Scaled X input of train set (matrix)
        y_train (Series):             y output to train on (vector)
    Returns:
        nn_regression_sets (array): true y values and predicted y values for train and validation set
    """
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, random_state=_random_state, test_size=_validation_size)
    neural_network = keras.Sequential(
        [layers.Dense(36, activation="relu", input_shape=[X_train_scaled.shape[1]], kernel_initializer="random_normal"),
         # layers.Dropout(0.2),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="relu", kernel_initializer="random_normal"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dense(36, activation="softmax"),
         # layers.Dropout(0.2),
         layers.Dense(1)])
    optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    neural_network.compile(loss="mse",
                           optimizer=optimizer,
                           metrics=["mae", "mse"])
    epochs = _epochs
    # create a validation set which is 20% of the whole dataset. Therefore use formula to receive ca. 0.2857.
    history = neural_network.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val))
    neural_network.save(os.path.join(path_output, "models", "NN_Regression_Model_"+on+"_"+hex_size))
    y_prediction_train = neural_network.predict(X_train)
    y_prediction_val = neural_network.predict(X_val)
    
    r2_train = r2_score(y_train, y_prediction_train)
    print("R2 (Train):",r2_train)
    r2_val = r2_score(y_val, y_prediction_val)
    print("R2 (Validation):",r2_val)
    
    plot_train_loss(history, on=on, hex_size=hex_size)
    nn_regression_sets = [y_train, y_val, y_prediction_train, y_prediction_val]
    return nn_regression_sets

### Loss visualization by epoch

In [7]:
def plot_train_loss(history, on, hex_size):
    """
    Plot the train and validation loss of Neural Network.

    Args:
        history (Object): History of loss during training of neural network
        on (str): time resolution to train on
    Returns:
        No return
    """
    print("Plot training and visualization loss...")
    # Plotting the training and validation loss
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs = range(1, len(loss) + 1)
    fig, ax = plt.subplots(figsize=(16, 8), dpi=300)
    ax.plot(epochs, loss, "bo", label="Training loss")
    ax.plot(epochs, val_loss, "b", label="Validation loss")
    ax.set_title("Training and validation loss "+on+"_"+hex_size, fontsize=18)
    ax.set_xlabel("Epochs", fontsize=16)
    ax.set_ylabel("Loss", fontsize=16)
    plt.legend()
    #plt.show()
    fig.savefig(os.path.join(path_output, "NN_error_per_epoch_"+on+"_"+hex_size+".png"))
    plt.close(fig)

### Run NN

In [8]:
def train_NN(on="24_demand", hex_size="hexa_big"):
    """
    Split the data in train and test set by 0.3 test set. 
    Then Scale the data and do a PCA. 
    Last train the NN on the chosen time resolution
    
    Args:
        on (str): time resolution to train on
        
    Returns:
        No return
    """
    print("Time Resolution is", on)
    #print("Split Data with random state", _random_state, "and test size", str(_test_size)+"...")
    # features_X = features.drop(["24_sum", "6_sum", "2_sum", "1_sum"], axis=1)
    features_X = features.drop(["24_demand", "24_demand_hex_big", "24_demand_hex_small", "24_agg_time",
                                "6_demand", "6_demand_hex_big", "6_demand_hex_small", "6_agg_time",
                                "2_demand", "2_demand_hex_big", "2_demand_hex_small", "2_agg_time",
                                "1_demand", "1_demand_hex_big", "1_demand_hex_small", "1_agg_time",
                                "24_available_hex_big"], axis=1)
    features_y = features[on]
    
    # Spatial Resolution
    print("Spatial Resolution is", hex_size)
    if hex_size=="hexa_small":
        features_X = features_X.drop("hexa_big", axis=1)
    else:
        features_X = features_X.drop("hexa_small", axis=1)
    
    #Split
    X_train, X_test, y_train, y_test = train_test_split(features_X, features_y, random_state=_random_state, test_size=_test_size)

    #print("Scale", hex_size, "Data with Standard Scaler...")
    with open(os.path.join(path_output, "models", "Standard_Scaler_"+hex_size+".pkl"), "rb") as f:
        standard_scaler = pickle.load(f)
    X_train_scaled = standard_scaler.transform(X_train)

    #print("Do PCA on", hex_size, "Data...")
    with open(os.path.join(path_output, "models", "PCA_"+hex_size+".pkl"), "rb") as f:
        pca = pickle.load(f)
    X_train_transformed = pca.transform(X_train_scaled)

    print("Train", "NN_Regression_Model_"+on+"_"+hex_size, "...")
    nn_regression_sets = train_neural_network(X_train_transformed, y_train.to_numpy(), on=on, hex_size=hex_size)

In [9]:
#Train the NN for each time resolution.
# hex_size = ["hexa_small", "hexa_big"]
hex_size = ["hexa_big", "hexa_small"]
time_resolutions = ["24_demand", "6_demand", "2_demand", "1_demand"]

"""
for time in time_resolutions:
    for size in hex_size:
        train_NN(on=time, hex_size=size)
        print()
print("Done")
"""
train_NN(on="24_available_hex_big", hex_size="hexa_big")

Time Resolution is 24_available_hex_big
Spatial Resolution is hexa_big
Train NN_Regression_Model_24_available_hex_big_hexa_big ...
Epoch 1/20
8867/8867 [==============================] - 13s 2ms/step - loss: 20088.3008 - mae: 118.5551 - mse: 20088.3008 - val_loss: 18588.7285 - val_mae: 116.3619 - val_mse: 18588.7285
Epoch 2/20
8867/8867 [==============================] - 12s 1ms/step - loss: 18175.3242 - mae: 113.0830 - mse: 18175.3242 - val_loss: 17396.5469 - val_mae: 110.0716 - val_mse: 17396.5469
Epoch 3/20
8867/8867 [==============================] - 12s 1ms/step - loss: 16937.5059 - mae: 107.6971 - mse: 16937.5059 - val_loss: 16750.1719 - val_mae: 106.1913 - val_mse: 16750.1719
Epoch 4/20
8867/8867 [==============================] - 13s 1ms/step - loss: 15995.8574 - mae: 103.9586 - mse: 15995.8574 - val_loss: 15902.2354 - val_mae: 103.2684 - val_mse: 15902.2354
Epoch 5/20
8867/8867 [==============================] - 13s 1ms/step - loss: 15242.7139 - mae: 99.8790 - mse: 15242.7139 